In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score, recall_score, precision_score, roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from scipy.linalg import sqrtm

# 1. FD Score (Fréchet Distance) 계산 함수
def calculate_fd(real_data, synth_data):
    """실제 데이터와 가짜 데이터의 분포 유사도 측정 (낮을수록 좋음)"""
    mu1, sigma1 = real_data.mean(axis=0), np.cov(real_data, rowvar=False)
    mu2, sigma2 = synth_data.mean(axis=0), np.cov(synth_data, rowvar=False)
    
    # 평균 차이의 제곱
    diff = mu1 - mu2
    # 공분산 행렬의 곱의 제곱근 계산
    covmean, _ = sqrtm(sigma1.dot(sigma2), disp=False)
    if np.iscomplexobj(covmean):
        covmean = covmean.real
        
    fd = diff.dot(diff) + np.trace(sigma1 + sigma2 - 2.0 * covmean)
    return fd

# 2. 성능 비교 메인 함수
def evaluate_all_methods(real_csv, smote_csv, cgan_csv, kcgan_csv, test_csv):
    # 테스트 데이터 로드 (실제 데이터로만 구성된 테스트셋)
    test_df = pd.read_csv(test_csv)
    X_test = test_df.drop('Class', axis=1)
    y_test = test_df['Class']

    paths = {
        'SMOTE': smote_csv,
        'Basic cGAN': cgan_csv,
        'K-cGAN + KL Loss': kcgan_csv
    }
    
    # 실제 사기 데이터 (FD 계산용)
    real_df = pd.read_csv(real_csv)
    real_fraud = real_df[real_df['Class'] == 1].drop('Class', axis=1)

    results = []

    for name, path in paths.items():
        print(f"📊 {name} 성능 측정 중...")
        train_df = pd.read_csv(path)
        X_train = train_df.drop('Class', axis=1)
        y_train = train_df['Class']
        
        # [A] FD Score 계산 (사기 데이터만 비교)
        synth_fraud = train_df[train_df['Class'] == 1].drop('Class', axis=1)
        # 차원이 맞지 않는 경우를 대비해 샘플링
        n = min(len(real_fraud), len(synth_fraud))
        fd_score = calculate_fd(real_fraud.iloc[:n], synth_fraud.iloc[:n])
        
        # [B] 분류 성능 측정 (Random Forest)
        clf = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)
        
        # 지표 계산
        results.append({
            'Method': name,
            'FD Score (↓)': round(fd_score, 2),
            'F1-Score (↑)': round(f1_score(y_test, y_pred), 4),
            'Recall (↑)': round(recall_score(y_test, y_pred), 4),
            'ROC-AUC (↑)': round(roc_auc_score(y_test, y_pred), 4)
        })

    # 결과 테이블 출력
    performance_table = pd.DataFrame(results)
    print("\n" + "="*60)
    print("           Final Performance Comparison Table")
    print("="*60)
    print(performance_table.to_string(index=False))
    print("="*60)
    
    # CSV로 저장
    performance_table.to_csv("performance_comparison.csv", index=False)
    return performance_table

# 실행
if __name__ == "__main__":
    # 파일명이 본인의 파일명과 일치하는지 확인하세요.
    evaluate_all_methods(
        real_csv='base_preprocessed.csv',
        smote_csv='train_smote.csv',
        cgan_csv='train_cgan.csv',
        kcgan_csv='train_kcgan.csv',
        test_csv='test_kcgan.csv' # 이전에 분리해둔 순수 실제 테스트셋
    )

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import f1_score, recall_score, roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler
from scipy.linalg import sqrtm

# ==========================================
# 1. 유틸리티 함수 (FD Score 계산)
# ==========================================
def calculate_fd(real_data, synth_data):
    """Fréchet Distance: 실제와 가짜 데이터의 분포 거리 (낮을수록 좋음)"""
    mu1, sigma1 = real_data.mean(axis=0), np.cov(real_data, rowvar=False)
    mu2, sigma2 = synth_data.mean(axis=0), np.cov(synth_data, rowvar=False)
    
    diff = mu1 - mu2
    covmean, _ = sqrtm(sigma1.dot(sigma2), disp=False)
    if np.iscomplexobj(covmean):
        covmean = covmean.real
        
    fd = diff.dot(diff) + np.trace(sigma1 + sigma2 - 2.0 * covmean)
    return fd

# ==========================================
# 2. 통합 실행 함수 (계산 + 시각화)
# ==========================================
def run_evaluation_and_visualization(real_path, smote_path, cgan_path, kcgan_path, test_path):
    print("🚀 성능 평가 및 시각화 통합 프로세스 시작...")
    
    # 데이터 로드
    paths = {
        'SMOTE': smote_path,
        'Basic cGAN': cgan_path,
        'K-cGAN + KL (Ours)': kcgan_path
    }
    
    real_df = pd.read_csv(real_path)
    real_fraud = real_df[real_df['Class'] == 1].drop('Class', axis=1)
    
    test_df = pd.read_csv(test_path)
    X_test = test_df.drop('Class', axis=1)
    y_test = test_df['Class']
    
    # 결과를 저장할 딕셔너리
    metrics_summary = {}

    # --------------------------------------
    # Phase 1: 성능 지표 계산 (Metrics)
    # --------------------------------------
    print("\n[Phase 1] 정량적 성능 지표 계산 중...")
    
    results_list = []
    
    for name, path in paths.items():
        print(f"  👉 Measuring {name}...")
        train_df = pd.read_csv(path)
        X_train = train_df.drop('Class', axis=1)
        y_train = train_df['Class']
        
        # 1. FD Score (사기 데이터 간 분포 거리)
        synth_fraud = train_df[train_df['Class'] == 1].drop('Class', axis=1)
        n_min = min(len(real_fraud), len(synth_fraud))
        fd = calculate_fd(real_fraud.iloc[:n_min], synth_fraud.iloc[:n_min])
        
        # 2. 분류 성능 (Random Forest)
        clf = RandomForestClassifier(n_estimators=50, random_state=42, n_jobs=-1)
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)
        
        f1 = f1_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        
        # 결과 저장
        metrics_summary[name] = {'FD': fd, 'F1': f1, 'Recall': recall}
        results_list.append({
            'Method': name, 'FD Score': round(fd, 2), 
            'F1-Score': round(f1, 4), 'Recall': round(recall, 4)
        })

    # 표 출력 및 저장
    res_df = pd.DataFrame(results_list)
    print("\n" + "="*50)
    print("       Final Performance Table")
    print("="*50)
    print(res_df.to_string(index=False))
    print("="*50)
    res_df.to_csv("performance_comparison.csv", index=False)

    # --------------------------------------
    # Phase 2: 시각화 (t-SNE Plot with Metrics)
    # --------------------------------------
    print("\n[Phase 2] t-SNE 분포 시각화 생성 중...")
    
    # 샘플링 (속도 및 시각적 균형을 위해 800개로 제한)
    n_samples = 800
    real_sample = real_fraud.sample(n=min(len(real_fraud), n_samples), random_state=42)
    
    # 캔버스 설정
    fig, axes = plt.subplots(1, 3, figsize=(22, 7))
    # 전체 제목에 실험 요약
    fig.suptitle('Comparison of Fraud Data Distribution & Detection Performance', fontsize=22, weight='bold', y=1.05)
    
    colors = {'Real': '#1f77b4', 'Synthetic': '#ff7f0e'} # 파랑(Real), 주황(Fake)

    for i, (name, path) in enumerate(paths.items()):
        # 데이터 준비
        df = pd.read_csv(path)
        synth_fraud = df[df['Class'] == 1].drop('Class', axis=1)
        synth_sample = synth_fraud.sample(n=min(len(synth_fraud), n_samples), random_state=42)
        
        # t-SNE 계산
        combined = pd.concat([real_sample, synth_sample])
        scaled = StandardScaler().fit_transform(combined)
        tsne = TSNE(n_components=2, random_state=42, perplexity=30)
        res = tsne.fit_transform(scaled)
        
        # Plot용 DataFrame
        plot_df = pd.DataFrame(res, columns=['x', 'y'])
        plot_df['Label'] = ['Real Fraud'] * len(real_sample) + ['Synthetic Fraud'] * len(synth_sample)
        
        # 서브플롯 그리기
        ax = axes[i]
        sns.scatterplot(
            data=plot_df, x='x', y='y', hue='Label',
            palette=[colors['Real'], colors['Synthetic']],
            alpha=0.6, s=50, edgecolor='w', ax=ax
        )
        
        # ★ 핵심: 제목에 점수 포함시키기 ★
        score_info = metrics_summary[name]
        title_text = (f"{name}\n"
                      f"FD: {score_info['FD']:.2f} (↓) | "
                      f"F1: {score_info['F1']:.3f} (↑)")
        
        ax.set_title(title_text, fontsize=16, weight='bold', pad=15)
        ax.legend(loc='upper right')
        ax.grid(True, linestyle='--', alpha=0.3)
        ax.set_xlabel('')
        ax.set_ylabel('')

    plt.tight_layout()
    plt.savefig('final_comparison_result.png', dpi=300, bbox_inches='tight')
    print("✅ 시각화 완료: final_comparison_result.png")
    plt.show()

# ==========================================
# 실행부
# ==========================================
if __name__ == "__main__":
    # 파일 경로가 맞는지 꼭 확인하세요!
    run_evaluation_and_visualization(
        real_path='base_preprocessed.csv',
        smote_path='train_smote.csv',
        cgan_path='train_cgan.csv',
        kcgan_path='train_kcgan.csv',
        test_path='test_kcgan.csv'
    )